## Importando bibliotecas

In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import adjusted_rand_score, adjusted_mutual_info_score
import seaborn as sns
from sklearn.preprocessing import StandardScaler

## Tratamento dos dados

In [2]:
df = sns.load_dataset('iris')
df["species"].replace({"setosa": 0, "versicolor": 1, "virginica": 2}, inplace=True)
df.columns = ["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm", "Class"]
labels = df["Class"].values
df.drop("Class", axis=1, inplace=True)
dados = df.to_numpy()

/tmp/ipykernel_13448/945038251.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["species"].replace({"setosa": 0, "versicolor": 1, "virginica": 2}, inplace=True)
/tmp/ipykernel_13448/945038251.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["species"].replace({"setosa": 0, "versicolor": 

## Método de agrupamento

In [3]:
class MFCM():
    def __init__(self, c, X, m):
        self.c = c
        self.n = X.shape[0]
        self.p = X.shape[1]
        self.m = m

        ##
        self.global_var = np.var(X, axis=0).mean() # pré-calcula a variância global dos dados para regularização

    def initialize_u(self):
        u_flat = np.random.dirichlet(alpha=np.ones(self.c * self.p), size=self.n)
        return u_flat.reshape(self.n, self.c, self.p)
    
    def initialize_lambda(self):
        return np.ones((self.c, self.p))
    
    def find_centroides(self, X, U):
        u_m = U ** self.m
        numerador = np.sum(u_m * X[:, np.newaxis, :], axis=0)
        denominador = np.sum(u_m, axis=0)
        denominador = np.fmax(denominador, np.finfo(np.float64).eps)
        return numerador / denominador
    
    def get_distances(self, X, V):
        return (X[:, np.newaxis, :] - V[np.newaxis, :, :]) ** 2

    def update_u(self, D, Lambda):
        power = 1.0 / (self.m - 1)
        eps = np.finfo(np.float64).eps
        
        weighted_dist = D * Lambda
        weighted_dist = np.fmax(weighted_dist, eps) 
        
        term = weighted_dist ** (-power)
        
        denominator = np.sum(term, axis=(1, 2), keepdims=True)
        denominator = np.fmax(denominator, eps)
        
        return term / denominator
    
    def update_lambda(self, D, U):
        eps = np.finfo(np.float64).eps
        
        term_k = (U ** self.m) * np.fmax(D, eps)
        S_ij = np.sum(term_k, axis=0) 
        
        # Regularização Aditiva
        ##
        regularization_factor = 0.01 * self.global_var
        S_ij = S_ij + regularization_factor 
        
        prod_S = np.prod(S_ij, axis=1, keepdims=True)
        numerator = prod_S ** (1.0 / self.p)
        
        Lambda = numerator / S_ij
        
        return Lambda

    ##
    def calculate_objective_function(self, D, U, Lambda):
        term = (U ** self.m) * D
        sum_k = np.sum(term, axis=0)
        return np.sum(Lambda * sum_k)

## Clustering

In [4]:
def mfcm_run(dados, num_clusters, m=2, max_iter=1000, epsilon=1e-6):
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(dados)
    
    mfcm = MFCM(c=num_clusters, X=X_scaled, m=m)
    
    indices = np.random.choice(X_scaled.shape[0], num_clusters, replace=False)
    centroids = X_scaled[indices]
    
    Lambda = np.ones((num_clusters, X_scaled.shape[1]))
    
    D = mfcm.get_distances(X_scaled, centroids)
    U = mfcm.update_u(D, Lambda)
    
    WARM_UP_ITERS = 20 

    for i in range(max_iter):
        U_old = U.copy()
        
        centroids = mfcm.find_centroides(X_scaled, U)
        D = mfcm.get_distances(X_scaled, centroids)
        
        ##
        if i >= WARM_UP_ITERS:
            Lambda = mfcm.update_lambda(D, U) 
        
        U = mfcm.update_u(D, Lambda)
        
        if np.linalg.norm(U - U_old) < epsilon:
            break
        
    J_final = mfcm.calculate_objective_function(D, U, Lambda)
    Delta = np.sum(U, axis=2)
    
    return centroids, U, Delta, Lambda, J_final

## Simulação de Monte Carlo

In [5]:
def calculate_objective_function(self, D, U, Lambda):
        term = (U ** self.m) * D
        sum_k = np.sum(term, axis=0)
        weighted_sum = Lambda * sum_k
        J = np.sum(weighted_sum)
        return J

In [6]:
def run_final_experiment(dados, labels, num_clusters=3, num_trials=100, restarts=50):
    results_ari = []
    results_ami = []
    
    for t in range(num_trials):
        best_J = np.inf
        best_pred = None
        
        for r in range(restarts):
            try:
                centroids, U, Delta, Lambda, J = mfcm_run(dados, num_clusters, m=2)
                
                if J > 1e-3 and J < best_J:
                    best_J = J
                    best_pred = np.argmax(Delta, axis=1)
            except:
                continue
        
        if best_pred is not None:
            ari = adjusted_rand_score(labels, best_pred)
            ami = adjusted_mutual_info_score(labels, best_pred)
            
            results_ari.append(ari)
            results_ami.append(ami)
            
            print(f"Trial {t+1}: J={best_J:.4f} | ARI={ari:.4f} | AMI={ami:.4f}")
        else:
            print(f"Trial {t+1}: Falha (Singularidade em todas as tentativas).")

    print(f"Mean ARI: {np.mean(results_ari):.4f} +/- {np.std(results_ari):.4f}")
    print(f"Mean AMI: {np.mean(results_ami):.4f} +/- {np.std(results_ami):.4f}")

In [7]:
run_final_experiment(dados, labels, num_clusters=3, num_trials=100, restarts=50)

Trial 1: J=0.0645 | ARI=0.2682 | AMI=0.2752
Trial 2: J=0.0637 | ARI=0.3246 | AMI=0.3532
Trial 3: J=0.0708 | ARI=0.3639 | AMI=0.3951
Trial 4: J=0.0591 | ARI=0.2170 | AMI=0.2288
Trial 5: J=0.0634 | ARI=0.2825 | AMI=0.2975
Trial 6: J=0.0619 | ARI=0.3112 | AMI=0.3171
Trial 7: J=0.0665 | ARI=0.2674 | AMI=0.2794
Trial 8: J=0.0626 | ARI=0.2846 | AMI=0.3027
Trial 9: J=0.0596 | ARI=0.2411 | AMI=0.2455
Trial 10: J=0.0608 | ARI=0.2527 | AMI=0.2739
Trial 11: J=0.0736 | ARI=0.2804 | AMI=0.2780
Trial 12: J=0.0642 | ARI=0.2175 | AMI=0.2596
Trial 13: J=0.0660 | ARI=0.2965 | AMI=0.3354
Trial 14: J=0.0662 | ARI=0.2799 | AMI=0.2957
Trial 15: J=0.0644 | ARI=0.2986 | AMI=0.3241
Trial 16: J=0.0589 | ARI=0.2064 | AMI=0.2053
Trial 17: J=0.0652 | ARI=0.2506 | AMI=0.2747
Trial 18: J=0.0662 | ARI=0.2846 | AMI=0.2814
Trial 19: J=0.0623 | ARI=0.2232 | AMI=0.2328
Trial 20: J=0.0643 | ARI=0.3768 | AMI=0.4324
Trial 21: J=0.0672 | ARI=0.3503 | AMI=0.3488
Trial 22: J=0.0613 | ARI=0.2709 | AMI=0.2877
Trial 23: J=0.0632 